# Imports

In [98]:
# !pip install pandas numpy scikit-learn statsmodels



In [99]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor


# Preprocessing

In [100]:
df = pd.read_csv('data\DOF_Condo_Income_NYC.csv')
df.head()

,Boro-Block-Lot,Condo Section,Address,Neighborhood,Building Classification,Total Units,Year Built,Gross SqFt,Estimated Gross Income,Gross Income per SqFt,...,Gross SqFt 3,Estimated Gross Income 3,Gross Income per SqFt 3,Estimated Expense 3,Expense per SqFt 3,Net Operating Income 3,Full Market Value 3,Market Value per SqFt 3,Distance from Condo in miles 2,Report Year
0,1-00576-7501,0003-R1,60 WEST 13 STREET,GREENWICH VILLAGE-CENTRAL,R4 -ELEVATOR,70,1966.0,82017,4452703,54.29,...,27108.0,1471693.0,54.29,571708.0,21.09,899985.0,7318000.0,269.96,0.35,2019
1,1-01271-7501,0007-R2,1360 6 AVENUE,MIDTOWN WEST,R4 -ELEVATOR,183,1963.0,141738,7113830,50.19,...,39510.0,1987748.0,50.31,527854.0,13.36,1459894.0,9322000.0,235.94,0.63,2019
2,1-00894-7501,0009-R1,77 PARK AVENUE,MURRAY HILL,R4 -ELEVATOR,109,1924.0,158571,7329152,46.22,...,20784.0,960636.0,46.22,374112.0,18.00,586524.0,3423000.0,164.69,0.22,2019
3,1-00631-7501,0018-R1,712 GREENWICH STREET,GREENWICH VILLAGE-WEST,R9 -CONDOPS,20,1910.0,53943,2132906,39.54,...,37064.0,1461804.0,39.44,504070.0,13.60,957734.0,14265000.0,384.87,0.19,2019
4,1-00868-7501,0019-R1,35 EAST 38 STREET,MURRAY HILL,R4 -ELEVATOR,113,1961.0,88230,4288860,48.61,...,41603.0,2022322.0,48.61,497572.0,11.96,1524750.0,7628000.0,183.35,0.22,2019


In [101]:
sorted_df = df[df['Boro-Block-Lot'] == '1-00576-7501'].sort_values('Report Year')[['Full Market Value', 'Report Year']]
sorted_df


,Full Market Value,Report Year
9691,16139988.0,2012
12089,15177002.0,2013
14994,15871998.0,2014
17819,17339995.0,2015
19637,18283995.0,2016
25627,20068999.0,2018
0,22115002.0,2019
2784,19402000.0,2020
5689,21920000.0,2021
28912,22058000.0,2023


In [102]:
df = df.drop(columns=['Address', 'Neighborhood','Condo Section', 'Address 1', 'Neighborhood 1','Distance from Condo in miles','Distance from Condo in miles 1', 'Distance from Condo in miles 2','Address 3', 'Neighborhood 3'])
df.columns

Index(['Boro-Block-Lot', 'Building Classification', 'Total Units',
       'Year Built', 'Gross SqFt', 'Estimated Gross Income',
       'Gross Income per SqFt', 'Estimated Expense', 'Expense per SqFt',
       'Net Operating Income', 'Full Market Value', 'Market Value per SqFt',
       'Boro-Block-Lot 1', 'Building Classification 1', 'Total Units 1',
       'Year Built 1', 'Gross SqFt 1', 'Estimated Gross Income 1',
       'Gross Income per SqFt 1', 'Estimated Expense 1', 'Expense per SqFt 1',
       'Net Operating Income 1', 'Full Market Value 1',
       'Market Value per SqFt 1', 'Boro-Block-Lot 2', 'Address 2',
       'Neighborhood 2', 'Building Classification 2', 'Total Units 2',
       'Year Built 2', 'Gross SqFt 2', 'Estimated Gross Income 2',
       'Gross Income per SqFt 2', 'Estimated Expense 2', 'Expense per SqFt 2',
       'Net Operating Income 2', 'Full Market Value 2',
       'Market Value per SqFt 2', 'Boro-Block-Lot 3',
       'Building Classification 3', 'Total Units 3', 

In [103]:
df['Age'] = df['Report Year'] - df['Year Built']
df['Age 1'] = df['Report Year'] - df['Year Built 1']
df['Age 2'] = df['Report Year'] - df['Year Built 2']
df['Age 3'] = df['Report Year'] - df['Year Built 3']
df = df.drop(columns=['Year Built', 'Year Built 1', 'Year Built 2', 'Year Built 3','Report Year'])

In [104]:
df['Borough'] = df['Boro-Block-Lot'].str[:1]
df['Borough 1'] = df['Boro-Block-Lot 1'].str[:1]
df['Borough 2'] = df['Boro-Block-Lot 2'].str[:1]
df['Borough 3'] = df['Boro-Block-Lot 3'].str[:1]
df = df.drop(columns=['Boro-Block-Lot', 'Boro-Block-Lot 1', 'Boro-Block-Lot 2', 'Boro-Block-Lot 3'])

Borough: 1=Manhattan, 2=The Bronx, 3=Brooklyn, 4=Queens, 5 = Staten Island.

In [105]:
df['Building Classification'] = df['Building Classification'].str.extract(r'([A-Z][a-zA-Z0-9])')
df['Building Classification 1'] = df['Building Classification 1'].str.extract(r'([A-Z][a-zA-Z0-9])')
df['Building Classification 2'] = df['Building Classification 2'].str.extract(r'([A-Z][a-zA-Z0-9])')
df['Building Classification 3'] = df['Building Classification 3'].str.extract(r'([A-Z][a-zA-Z0-9])')

# If the condo classification is R2, make it Elevator. If it is R4, make it Walk-up
df['Has Elevator'] = np.where(df['Building Classification'] == 'R2', 1, 0)
# If the apartment bldg classification starts with D, make it elevator. If it starts with C, make it walk-up
df['Has Elevator 1'] = np.where(df['Building Classification 1'].str.startswith('D'), 1, 0)
df['Has Elevator 2'] = np.where(df['Building Classification 2'].str.startswith('D'), 1, 0)
df['Has Elevator 3'] = np.where(df['Building Classification 3'].str.startswith('D'), 1, 0)

df = df.drop(columns=['Building Classification', 'Building Classification 1', 'Building Classification 2', 'Building Classification 3'])

https://www.nyc.gov/assets/finance/jump/hlpbldgcode.html

In [106]:
df['Has Elevator'].value_counts()

Has Elevator
0    26583
1     4992
Name: count, dtype: int64

In [107]:
df.columns

Index(['Total Units', 'Gross SqFt', 'Estimated Gross Income',
       'Gross Income per SqFt', 'Estimated Expense', 'Expense per SqFt',
       'Net Operating Income', 'Full Market Value', 'Market Value per SqFt',
       'Total Units 1', 'Gross SqFt 1', 'Estimated Gross Income 1',
       'Gross Income per SqFt 1', 'Estimated Expense 1', 'Expense per SqFt 1',
       'Net Operating Income 1', 'Full Market Value 1',
       'Market Value per SqFt 1', 'Address 2', 'Neighborhood 2',
       'Total Units 2', 'Gross SqFt 2', 'Estimated Gross Income 2',
       'Gross Income per SqFt 2', 'Estimated Expense 2', 'Expense per SqFt 2',
       'Net Operating Income 2', 'Full Market Value 2',
       'Market Value per SqFt 2', 'Total Units 3', 'Gross SqFt 3',
       'Estimated Gross Income 3', 'Gross Income per SqFt 3',
       'Estimated Expense 3', 'Expense per SqFt 3', 'Net Operating Income 3',
       'Full Market Value 3', 'Market Value per SqFt 3', 'Age', 'Age 1',
       'Age 2', 'Age 3', 'Borough', '

In [108]:
condo_columns = ['Borough', 'Total Units', 'Age', 'Has Elevator', 'Gross SqFt', 'Estimated Gross Income',
                 'Estimated Expense','Net Operating Income', 'Full Market Value']

apt_columns_1 = ['Borough 1', 'Total Units 1', 'Age 1', 'Has Elevator 1', 'Gross SqFt 1', 'Estimated Gross Income 1',
                 'Estimated Expense 1','Net Operating Income 1', 'Full Market Value 1']

apt_columns_2 = ['Borough 2', 'Total Units 2', 'Age 2', 'Has Elevator 2', 'Gross SqFt 2', 'Estimated Gross Income 2',
                 'Estimated Expense 2','Net Operating Income 2', 'Full Market Value 2']

apt_columns_3 = ['Borough 3', 'Total Units 3', 'Age 3', 'Has Elevator 3', 'Gross SqFt 3', 'Estimated Gross Income 3',
                 'Estimated Expense 3','Net Operating Income 3', 'Full Market Value 3']

# concatenate the lists
columns = condo_columns + apt_columns_1 + apt_columns_2 + apt_columns_3

In [109]:
# create a new dataframe with the columns
df_final = df[columns]
df_final

,Borough,Total Units,Age,Has Elevator,Gross SqFt,Estimated Gross Income,Estimated Expense,Net Operating Income,Full Market Value,Borough 1,...,Full Market Value 2,Borough 3,Total Units 3,Age 3,Has Elevator 3,Gross SqFt 3,Estimated Gross Income 3,Estimated Expense 3,Net Operating Income 3,Full Market Value 3
0,1,70,53.0,0,82017,4452703,1729739.0,2722964,22115002.0,1,...,6936000.0,1,35.0,119.0,1,27108.0,1471693.0,571708.0,899985.0,7318000.0
1,1,183,56.0,0,141738,7113830,2361355.0,4752475,38596999.0,1,...,9522000.0,1,48.0,77.0,1,39510.0,1987748.0,527854.0,1459894.0,9322000.0
2,1,109,95.0,0,158571,7329152,2854278.0,4474874,36343010.0,1,...,24802000.0,1,19.0,109.0,1,20784.0,960636.0,374112.0,586524.0,3423000.0
3,1,20,109.0,0,53943,2132906,666196.0,1466710,11912000.0,1,...,16016000.0,1,49.0,89.0,1,37064.0,1461804.0,504070.0,957734.0,14265000.0
4,1,113,58.0,0,88230,4288860,1055231.0,3233629,26261996.0,1,...,10386000.0,1,39.0,95.0,1,41603.0,2022322.0,497572.0,1524750.0,7628000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31570,5,52,35.0,0,63601,1425934,751764.0,674171,5267000.0,5,...,25158000.0,5,305.0,53.0,1,297600.0,6743616.0,3217056.0,3526560.0,26655000.0
31571,5,48,38.0,1,51425,1097410,552819.0,544591,4225000.0,5,...,1879000.0,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
31572,5,40,77.0,1,38700,846369,524772.0,321597,2505000.0,5,...,821000.0,5,14.0,49.0,0,8950.0,210862.0,74106.0,136756.0,1110000.0
31573,5,48,50.0,1,38424,745810,361186.0,384624,2938000.0,5,...,809000.0,5,15.0,48.0,0,10626.0,206251.0,99884.0,106366.0,813000.0


In [110]:
df_final.dtypes

Borough                      object
Total Units                   int64
Age                         float64
Has Elevator                  int32
Gross SqFt                    int64
Estimated Gross Income        int64
Estimated Expense           float64
Net Operating Income          int64
Full Market Value           float64
Borough 1                    object
Total Units 1               float64
Age 1                       float64
Has Elevator 1                int32
Gross SqFt 1                float64
Estimated Gross Income 1    float64
Estimated Expense 1         float64
Net Operating Income 1      float64
Full Market Value 1         float64
Borough 2                    object
Total Units 2               float64
Age 2                       float64
Has Elevator 2                int32
Gross SqFt 2                float64
Estimated Gross Income 2    float64
Estimated Expense 2         float64
Net Operating Income 2      float64
Full Market Value 2         float64
Borough 3                   

In [111]:
df_final = df_final.dropna()
df_final = df_final.drop_duplicates()

In [112]:
# make borough int dtype
df_final['Borough'] = df_final['Borough'].astype(int)
df_final['Borough 1'] = df_final['Borough 1'].astype(int)
df_final['Borough 2'] = df_final['Borough 2'].astype(int)
df_final['Borough 3'] = df_final['Borough 3'].astype(int)

# make total units int dtype
df_final['Total Units'] = df_final['Total Units'].astype(int)
df_final['Total Units 1'] = df_final['Total Units 1'].astype(int)
df_final['Total Units 2'] = df_final['Total Units 2'].astype(int)
df_final['Total Units 3'] = df_final['Total Units 3'].astype(int)

# make has elevator int dtype
df_final['Has Elevator'] = df_final['Has Elevator'].astype(int)
df_final['Has Elevator 1'] = df_final['Has Elevator 1'].astype(int)
df_final['Has Elevator 2'] = df_final['Has Elevator 2'].astype(int)
df_final['Has Elevator 3'] = df_final['Has Elevator 3'].astype(int)


In [113]:
# make gross sqft float dtype
df_final['Gross SqFt'] = df_final['Gross SqFt'].astype(float)
df_final['Gross SqFt 1'] = df_final['Gross SqFt 1'].astype(float)
df_final['Gross SqFt 2'] = df_final['Gross SqFt 2'].astype(float)
df_final['Gross SqFt 3'] = df_final['Gross SqFt 3'].astype(float)

# make estimated gross income float dtype
df_final['Estimated Gross Income'] = df_final['Estimated Gross Income'].astype(float)
df_final['Estimated Gross Income 1'] = df_final['Estimated Gross Income 1'].astype(float)
df_final['Estimated Gross Income 2'] = df_final['Estimated Gross Income 2'].astype(float)
df_final['Estimated Gross Income 3'] = df_final['Estimated Gross Income 3'].astype(float)

# make estimated expense float dtype
df_final['Estimated Expense'] = df_final['Estimated Expense'].astype(float)
df_final['Estimated Expense 1'] = df_final['Estimated Expense 1'].astype(float)
df_final['Estimated Expense 2'] = df_final['Estimated Expense 2'].astype(float)
df_final['Estimated Expense 3'] = df_final['Estimated Expense 3'].astype(float)

# make net operating income float dtype
df_final['Net Operating Income'] = df_final['Net Operating Income'].astype(float)
df_final['Net Operating Income 1'] = df_final['Net Operating Income 1'].astype(float)
df_final['Net Operating Income 2'] = df_final['Net Operating Income 2'].astype(float)
df_final['Net Operating Income 3'] = df_final['Net Operating Income 3'].astype(float)

# make full market value float dtype
df_final['Full Market Value'] = df_final['Full Market Value'].astype(float)
df_final['Full Market Value 1'] = df_final['Full Market Value 1'].astype(float)
df_final['Full Market Value 2'] = df_final['Full Market Value 2'].astype(float)
df_final['Full Market Value 3'] = df_final['Full Market Value 3'].astype(float)

In [114]:
df_final.dtypes

Borough                       int32
Total Units                   int32
Age                         float64
Has Elevator                  int32
Gross SqFt                  float64
Estimated Gross Income      float64
Estimated Expense           float64
Net Operating Income        float64
Full Market Value           float64
Borough 1                     int32
Total Units 1                 int32
Age 1                       float64
Has Elevator 1                int32
Gross SqFt 1                float64
Estimated Gross Income 1    float64
Estimated Expense 1         float64
Net Operating Income 1      float64
Full Market Value 1         float64
Borough 2                     int32
Total Units 2                 int32
Age 2                       float64
Has Elevator 2                int32
Gross SqFt 2                float64
Estimated Gross Income 2    float64
Estimated Expense 2         float64
Net Operating Income 2      float64
Full Market Value 2         float64
Borough 3                   

In [130]:
from scipy.stats import zscore

numeric_columns = ['Age', 'Gross SqFt', 'Estimated Gross Income', 'Estimated Expense', 
                      'Net Operating Income', 'Full Market Value', 'Age 1', 'Gross SqFt 1', 
                      'Estimated Gross Income 1', 'Estimated Expense 1', 'Net Operating Income 1', 
                      'Full Market Value 1', 'Age 2', 'Gross SqFt 2', 'Estimated Gross Income 2', 
                      'Estimated Expense 2', 'Net Operating Income 2', 'Full Market Value 2', 
                      'Age 3', 'Gross SqFt 3', 'Estimated Gross Income 3', 'Estimated Expense 3', 
                      'Net Operating Income 3', 'Full Market Value 3']

z_scores = np.abs(zscore(df_final[numeric_columns]))
mask = (z_scores < 2).all(axis=1)
df_final = df_final[mask]
df_final

,Borough,Total Units,Age,Has Elevator,Gross SqFt,Estimated Gross Income,Estimated Expense,Net Operating Income,Full Market Value,Borough 1,...,Full Market Value 2,Borough 3,Total Units 3,Age 3,Has Elevator 3,Gross SqFt 3,Estimated Gross Income 3,Estimated Expense 3,Net Operating Income 3,Full Market Value 3
0,4,32,10.0,0,78540.0,2238390.0,822314.0,1416076.0,11328004.0,4,...,5650000.0,4,95,60.0,1,82284.0,2351677.0,891959.0,1459718.0,4534000.0
1,3,8,8.0,0,7472.0,111183.0,37734.0,73449.0,496002.0,3,...,306000.0,3,16,67.0,0,11800.0,174286.0,50032.0,124254.0,667000.0
2,1,8,33.0,0,4227.0,199683.0,56938.0,142746.0,1159000.0,1,...,6466000.0,1,50,109.0,1,22500.0,998775.0,372375.0,626400.0,5074000.0
4,1,84,57.0,0,96152.0,3318206.0,1061518.0,2256688.0,17536000.0,1,...,14746000.0,1,90,63.0,1,70992.0,2280464.0,711786.0,1568678.0,11844000.0
5,3,13,13.0,0,20340.0,356357.0,121023.0,235334.0,1755001.0,3,...,926000.0,3,40,91.0,0,30800.0,539616.0,183260.0,356356.0,1605000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3,60,12.0,0,51102.0,1716516.0,602493.0,1114024.0,9067000.0,3,...,13545000.0,3,18,12.0,1,18900.0,478737.0,198828.0,279909.0,2242000.0
19996,1,59,91.0,0,71863.0,3348816.0,1328747.0,2020069.0,16274000.0,1,...,14413000.0,1,38,101.0,1,50592.0,2335833.0,892949.0,1442884.0,10561000.0
19997,3,36,13.0,1,52477.0,1620490.0,642843.0,977647.0,7820998.0,3,...,11079000.0,3,42,107.0,0,48980.0,1528176.0,702373.0,825803.0,6606000.0
19998,3,44,10.0,0,35216.0,1294892.0,340539.0,954354.0,7715000.0,3,...,1198000.0,3,30,107.0,1,25000.0,919250.0,241750.0,677500.0,1708000.0


In [131]:
df_final = df_final.sample(10_000, random_state=42).reset_index(drop=True)
df_final

,Borough,Total Units,Age,Has Elevator,Gross SqFt,Estimated Gross Income,Estimated Expense,Net Operating Income,Full Market Value,Borough 1,...,Full Market Value 2,Borough 3,Total Units 3,Age 3,Has Elevator 3,Gross SqFt 3,Estimated Gross Income 3,Estimated Expense 3,Net Operating Income 3,Full Market Value 3
0,3,12,113.0,1,13732.0,559167.0,190051.0,369116.0,2984000.0,3,...,749000.0,3,17,112.0,0,15120.0,610697.0,207295.0,403402.0,836000.0
1,1,55,97.0,0,80629.0,2900225.0,827254.0,2072971.0,16582001.0,1,...,7961000.0,1,60,91.0,1,49977.0,2212982.0,874098.0,1338884.0,10857000.0
2,3,25,18.0,0,43876.0,1010026.0,505013.0,505013.0,3961000.0,3,...,12754000.0,3,66,84.0,1,72000.0,1167840.0,609840.0,558000.0,4104000.0
3,3,53,106.0,1,63540.0,1629166.0,568048.0,1061118.0,8512000.0,3,...,2341000.0,3,30,110.0,0,21865.0,560619.0,195473.0,365146.0,3117000.0
4,1,13,14.0,0,37035.0,1637317.0,442198.0,1195119.0,9560003.0,1,...,2914000.0,1,96,40.0,1,57538.0,2543755.0,687004.0,1856751.0,16644000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5,57,18.0,0,62045.0,1512657.0,829542.0,683115.0,5392000.0,5,...,5612000.0,5,108,98.0,1,83646.0,2039289.0,1118347.0,920942.0,7017000.0
9996,3,10,12.0,0,11935.0,325348.0,75907.0,249441.0,2007999.0,3,...,8199000.0,3,49,89.0,1,60720.0,1636404.0,375250.0,1261154.0,5019000.0
9997,3,14,17.0,0,25384.0,656938.0,227948.0,428990.0,3444000.0,3,...,12422000.0,3,24,12.0,1,27797.0,702986.0,392216.0,310770.0,3173000.0
9998,1,20,118.0,0,11852.0,564748.0,188921.0,375827.0,3025000.0,1,...,13294000.0,1,31,98.0,1,19260.0,918124.0,285048.0,633076.0,5670000.0


In [132]:
train_data = df_final.sample(frac=0.8, random_state=42)
test_data = df_final.drop(train_data.index)
test_data = test_data.reset_index(drop=True)

In [133]:
train_data_condo = train_data[condo_columns]
test_data_condo = test_data[condo_columns]

In [119]:
train_data_condo.isna().sum()

Borough                   0
Total Units               0
Age                       0
Has Elevator              0
Gross SqFt                0
Estimated Gross Income    0
Estimated Expense         0
Net Operating Income      0
Full Market Value         0
dtype: int64

In [134]:
train_data.to_csv(r'data\train_data_all.csv',index=False)
test_data.to_csv(r'data\test_data_all.csv',index=False)
train_data_condo.to_csv(r'data\train_data_condo.csv',index=False)
test_data_condo.to_csv(r'data\test_data_condo.csv',index=False)

In [37]:
train_data_condo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16000 entries, 10650 to 5047
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Borough                 16000 non-null  int32  
 1   Total Units             16000 non-null  int32  
 2   Age                     16000 non-null  int32  
 3   Has Elevator            16000 non-null  int32  
 4   Gross SqFt              16000 non-null  int32  
 5   Estimated Gross Income  16000 non-null  float64
 6   Estimated Expense       16000 non-null  float64
 7   Net Operating Income    16000 non-null  float64
 8   Full Market Value       16000 non-null  float64
dtypes: float64(4), int32(5)
memory usage: 937.5 KB


In [81]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16000 entries, 10650 to 5047
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Borough                   16000 non-null  int32  
 1   Total Units               16000 non-null  int32  
 2   Age                       16000 non-null  int32  
 3   Has Elevator              16000 non-null  int32  
 4   Gross SqFt                16000 non-null  int32  
 5   Estimated Gross Income    16000 non-null  float64
 6   Estimated Expense         16000 non-null  float64
 7   Net Operating Income      16000 non-null  float64
 8   Full Market Value         16000 non-null  float64
 9   Borough 1                 16000 non-null  int32  
 10  Total Units 1             16000 non-null  int32  
 11  Age 1                     16000 non-null  int32  
 12  Has Elevator 1            16000 non-null  int32  
 13  Gross SqFt 1              16000 non-null  int32  
 14  Estimate